In [1]:
%%capture
!pip install datasets
!pip install transformers==4.28.0
!pip install torchaudio
!pip install jiwer

In [2]:
from huggingface_hub import notebook_login

notebook_login()


### Create `Wav2Vec2CTCTokenizer`

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import zipfile
from google.colab import output
from google.colab import drive
import os
import cv2 as cv
import numpy as np
from sklearn.model_selection import train_test_split

In [5]:
DATASET_PATH = '/content/drive/MyDrive/Datasets array form/ArrayForm_dataset.zip'
zip_object = zipfile.ZipFile(file =DATASET_PATH ,mode = 'r')
zip_object.extractall('./')
zip_object.close

<bound method ZipFile.close of <zipfile.ZipFile filename='/content/drive/MyDrive/Datasets array form/ArrayForm_dataset.zip' mode='r'>>

In [6]:
from datasets import load_dataset, load_metric, load_from_disk

data = load_from_disk("/content/ArrayForm_dataset")
train_data = data["train"].select(range(4500))  # Select the first 1000 examples
test_data = data["test"].select(range(500))  # Select the first 200 examples

In [7]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=3):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [8]:
show_random_elements(train_data, num_examples=5)

,path,sentence,speech,sampling_rate,target_text
0,D:/Ar-DAD Arabic Diversified Audio Dataset/Ar-DAD Arabic Diversified Audio Dataset/Audio_Dataset/Reciters/093/004/093004_27.wav,وَلَلْآخِرَةُ خَيْرٌ لَكَ مِنَ الْأُولَى,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]",16000,وَلَلْآخِرَةُ خَيْرٌ لَكَ مِنَ الْأُولَى
1,D:/Ar-DAD Arabic Diversified Audio Dataset/Ar-DAD Arabic Diversified Audio Dataset/Audio_Dataset/Reciters/084/002/084002_23.wav,وَأَذِنَتْ لِرَبِّهَا وَحُقَّتْ,"[-1.3658063835464418e-06, 3.198173999408027e-06, -2.1349020244088024e-05, -3.774075594265014e-06, 5.60732587473467e-07, 1.8692162484512664e-06, -1.0638555977493525e-05, -1.7824204405769706e-05, 3.33464122377336e-05, 2.6956609872286208e-05, 3.421601286390796e-05, 2.5653967895777896e-05, 4.750244988827035e-05, 8.282360795419663e-05, 3.022350210812874e-05, 3.162231951137073e-05, 2.7782341931015253e-05, 4.4401669583749026e-05, 4.4804350181948394e-05, -8.901478395273443e-06, 2.2400472516892478e-05, 5.293784397508716e-06, -1.7191678125527687e-07, -1.4928555174265057e-06, 3.4592826523294207e-06, -1.4506512343359645e-05, -3.273413676652126e-05, 1.497159701102646e-06, -2.1784740056318697e-06, 3.733051698873169e-06, -1.3389752893999685e-05, -1.5035662727314048e-05, 3.457540151430294e-05, 2.5691271730465814e-05, 3.8078655052231625e-05, 4.2749466956593096e-05, -7.200918844318949e-06, 5.658705504174577e-06, -7.20445405022474e-06, 2.6383335352875292e-05, -7.418137101922184e-06, -1.6120589862111956e-05, -2.122719342878554e-05, -6.030488293617964e-05, -4.743400040752022e-06, -4.33783952757949e-06, -3.2429310522275046e-05, -2.8204831323819235e-05, -3.492454197839834e-05, -1.219765363202896e-05, -2.2443095076596364e-05, -2.5745259335963055e-05, 7.3948012868640944e-06, -2.898040838772431e-05, 8.88204431248596e-06, -2.9211609216872603e-05, 1.6087995390989818e-05, 5.1008028094656765e-05, 2.5250546968891285e-05, 6.792596832383424e-05, 5.3968160500517115e-05, 4.621344123734161e-05, 4.379163874546066e-05, 3.3074702514568344e-05, 4.9928639782592654e-05, 4.36308364442084e-05, 6.702274549752474e-05, 5.841488018631935e-06, -3.5763674532063305e-05, -3.467520582489669e-05, -0.00010511170694371685, -5.911446351092309e-05, -0.00010175358329433948, -6.39346253592521e-05, -5.178593346499838e-05, -6.983200728427619e-05, -8.936786616686732e-05, -7.011673005763441e-05, -6.9967527451808564e-06, -6.16258112131618e-05, -4.363303742138669e-05, -2.042013147729449e-05, -7.965086115291342e-05, 4.7408524551428854e-05, 4.293735037208535e-05, 0.00010872863640543073, 0.00026519803213886917, 0.00028367957565933466, 0.00040873244870454073, 0.00038241280708462, 0.00031183293322101235, 0.00026325686485506594, 0.00015404255827888846, -6.90663728164509e-05, -0.00023075289209373295, -0.0003325233410578221, -0.0003535326395649463, -0.00041298929136246443, -0.0005421754904091358, -0.0006592199206352234, ...]",16000,وَأَذِنَتْ لِرَبِّهَا وَحُقَّتْ
2,D:/Ar-DAD Arabic Diversified Audio Dataset/Ar-DAD Arabic Diversified Audio Dataset/Audio_Dataset/Reciters/108/001/108001_24.wav,إِنَّا أَعْطَيْنَاكَ الْكَوْثَرَ,"[1.4998041697467102e-10, 1.052727002792686e-10, 9.718102988998112e-10, 1.5338064152103925e-11, 3.9674839802383843e-11, 6.508926037041007e-11, 2.360870388073977e-10, 3.759141886661155e-10, -7.326287493336636e-11, 5.266189706532032e-10, 2.152217015716218e-10, 6.523257489732259e-10, 2.197739212839167e-10, -1.5956448662368672e-10, -2.987787239838724e-10, -3.2397776128512135e-10, -3.741418980185429e-11, -2.6466637215172284e-10, 9.234647768696647e-11, 7.2001504758389

In [9]:
import IPython.display as ipd
import random
import numpy as np
rand_int = random.randint(0, len(train_data))

ipd.Audio(data=np.asarray(train_data[rand_int]["speech"]), autoplay=True, rate=16000)

In [10]:
print("Target text:", train_data[rand_int]["target_text"])
print("Input array shape:", np.asarray(train_data[rand_int]["speech"]).shape)
print("Sampling rate:", train_data[rand_int]["sampling_rate"])

Target text: كَلَّا سَيَعْلَمُونَ 
Input array shape: (61858,)
Sampling rate: 16000


### Create `Wav2Vec2FeatureExtractor`

In [11]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower() + " "
    return batch

In [12]:
train_data = train_data.map(remove_special_characters)
test_data = test_data.map(remove_special_characters)

Map:   0%|          | 0/4500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [13]:
show_random_elements(train_data.remove_columns(["path", "speech", "sampling_rate","target_text"]), num_examples=10)

,sentence
0,أَفَلَا يَعْلَمُ إِذَا بُعْثِرَ مَا فِي الْقُبُورِ
1,وَبَنَيْنَا فَوْقَكُمْ سَبْعًا شِدَادًا
2,وَهُوَ يَخْشَى
3,إِذَا السَّمَاءُ انْشَقَّتْ
4,وَالْفَجْرِ
5,وَكَذَّبُوا بِآيَاتِنَا كِذَّابًا
6,فَالْيَوْمَ الَّذِينَ آمَنُوا مِنَ الْكُفَّارِ يَضْحَكُونَ
7,فَأَمَّا الْإِنْسَانُ إِذَا مَا ابْتَلَاهُ رَبُّهُ فَأَكْرَمَهُ وَنَعَّمَهُ فَيَقُولُ رَبِّي أَكْرَمَنِ
8,إِنَّ الَّذِينَ كَفَرُوا مِنْ أَهْلِ الْكِتَابِ وَالْمُشْرِكِينَ فِي نَارِ جَهَنَّمَ خَالِدِينَ فِيهَا أُولَئِكَ هُمْ شَرُّ الْبَرِيَّةِ
9,إِذَا زُلْزِلَتِ الْأَرْضُ زِلْزَالَهَا


In [14]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [15]:
vocab_train = train_data.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=train_data.column_names)
vocab_test = test_data.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=test_data.column_names)

Map:   0%|          | 0/4500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [16]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [17]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'ا': 0,
 'ء': 1,
 'ى': 2,
 'ذ': 3,
 'ق': 4,
 'ً': 5,
 'ب': 6,
 'ِ': 7,
 'خ': 8,
 'ر': 9,
 'ه': 10,
 'م': 11,
 'ف': 12,
 'ظ': 13,
 'ض': 14,
 'ٌ': 15,
 'َ': 16,
 'ك': 17,
 'ْ': 18,
 'د': 19,
 'ة': 20,
 'إ': 21,
 'ص': 22,
 ' ': 23,
 'ل': 24,
 'غ': 25,
 'ح': 26,
 'ع': 27,
 'ي': 28,
 'ُ': 29,
 'و': 30,
 'أ': 31,
 'ت': 32,
 'ن': 33,
 'ث': 34,
 'ز': 35,
 'ئ': 36,
 'ؤ': 37,
 'ّ': 38,
 'ط': 39,
 'س': 40,
 'ٍ': 41,
 'آ': 42,
 'ش': 43,
 'ج': 44}

In [18]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [19]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

47

In [20]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [21]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [36]:
repo_name = "zarko1231/model-baru-collab-6"

In [37]:
tokenizer.push_to_hub(repo_name)

CommitInfo(commit_url='https://huggingface.co/zarko1231/model-baru-collab-6/commit/1a7322d45f8259d225de17a7cb871390f5bec6f1', commit_message='Upload tokenizer', commit_description='', oid='1a7322d45f8259d225de17a7cb871390f5bec6f1', pr_url=None, pr_revision=None, pr_num=None)

In [24]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [25]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer,)

In [26]:
def prepare_dataset(batch):
    audio = batch
    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["speech"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    batch["labels"] = processor(text=batch["sentence"], return_tensors="pt").input_ids[0]
    return batch

In [27]:
train_data = train_data.map(prepare_dataset, remove_columns=train_data.column_names,num_proc=4)
test_data = test_data.map(prepare_dataset, remove_columns=test_data.column_names,num_proc=4)

Map (num_proc=4):   0%|          | 0/4500 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/500 [00:00<?, ? examples/s]

In [28]:
max_input_length_in_sec = 6.0
train_data = train_data.filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

Filter:   0%|          | 0/4500 [00:00<?, ? examples/s]

In [29]:
test_data = test_data.filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

In [30]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [31]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [32]:
wer_metric = load_metric("wer")

<ipython-input-32-605de035dfd7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")


In [33]:
import numpy as np
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [38]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "zarko1231/model-baru-collab", 
    attention_dropout=0.3,
    hidden_dropout=0.2,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ignore_mismatched_sizes=True,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)

In [39]:
!pip install git-lfs


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [40]:
model.freeze_feature_encoder()

In [41]:
print(model)

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (1-4): 4 x Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projec

In [43]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=50,
  #auto_find_batch_size = True,
  gradient_accumulation_steps=2,
  logging_strategy='epoch',
  evaluation_strategy="epoch",
  num_train_epochs=25,
  gradient_checkpointing=True,
  fp16=True,
  learning_rate=2e-4,
  save_total_limit=1,
  save_strategy="epoch",
  push_to_hub=True,
  optim = "adamw_torch"
)


In [44]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=test_data,
    tokenizer=processor.feature_extractor,
    
)

Cloning https://huggingface.co/zarko1231/model-baru-collab-6 into local empty directory.


```javascript
function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}
setInterval(ConnectButton,60000);
```

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Wer
1,3.822900,2.036937,1.047724
2,1.011200,0.161243,0.500000
3,0.225500,0.093763,0.366373
4,0.194500,0.092610,0.358297
5,0.132900,0.081145,0.342878
6,0.109500,0.079725,0.333333
7,0.103400,0.085686,0.331865
8,0.098700,0.079209,0.329662
9,0.106800,0.081487,0.327460
10,0.087500,0.081098,0.325257


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call

In [ ]:
# Load the pytorch_model.bin file into a torch.nn.Module object.
model = torch.load('/content/drive/MyDrive/model ayat/pytorch_model.bin')

FileNotFoundError: ignored

In [ ]:
import gc
gc.collect()
#do below before defining model and trainer if you change batch size etc
#del trainer
#del model
torch.cuda.empty_cache()

In [ ]:
trainer.push_to_hub(repo_name)

Upload file pytorch_model.bin:   0%|          | 1.00/1.18G [00:00<?, ?B/s]

Upload file runs/Jun06_03-54-47_96ea7db5b33c/events.out.tfevents.1686023783.96ea7db5b33c.537.0:   0%|         …

To https://huggingface.co/zarko1231/model-baru-collab-4
   d46ca2a..ce5ea1d  main -> main

   d46ca2a..ce5ea1d  main -> main

To https://huggingface.co/zarko1231/model-baru-collab-4
   ce5ea1d..180626e  main -> main

   ce5ea1d..180626e  main -> main



'https://huggingface.co/zarko1231/model-baru-collab-4/commit/ce5ea1d3ab3eb906615793403d6fae68e32f1a21'

In [ ]:
model.push_to_hub(repo_name)

```python
from transformers import AutoModelForCTC, Wav2Vec2Processor

model = AutoModelForCTC.from_pretrained("patrickvonplaten/wav2vec2-large-xls-r-300m-tr-colab")
processor = Wav2Vec2Processor.from_pretrained("patrickvonplaten/wav2vec2-large-xls-r-300m-tr-colab")
```

For more examples of how XLS-R can be fine-tuned, please take a look at the [official speech recognition examples](https://github.com/huggingface/transformers/tree/master/examples/pytorch/speech-recognition#examples).

### Evaluation

As a final check, let's load the model and verify that it indeed has learned to transcribe Turkish speech.

Let's first load the pretrained checkpoint.

In [ ]:
model = Wav2Vec2ForCTC.from_pretrained(repo_name).to("cuda")
processor = Wav2Vec2Processor.from_pretrained(repo_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.



Now, we will just take the first example of the test set, run it through the model and take the `argmax(...)` of the logits to retrieve the predicted token ids.

In [ ]:
input_dict = processor(test_data["input_values"], return_tensors="pt", padding=True)

logits = model(input_dict.input_values.to("cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)[0]

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


OutOfMemoryError: ignored

We adapted `common_voice_test` quite a bit so that the dataset instance does not contain the original sentence label anymore. Thus, we re-use the original dataset to get the label of the first example.

In [ ]:
#common_voice_test_transcription = load_dataset("csv", data_dir="./cv-corpus-6.1-2020-12-11", split="test")
test_transcription=load_from_disk("/content/ArrayForm_dataset")
data_test_txt = test_transcription["test"].select(range(100))


Finally, we can decode the example.

In [ ]:
print("Prediction:")
print(processor.decode(pred_ids))

print("\nReference:")
print(data_test_txt[2]["sentence"].lower())

Prediction:
يَعْلَمُونَ مَا تَفْعَلّنَ

Reference:
وَلَا تَحَاضُّونَ عَلَى طَعَامِ الْمِسْكِينِ 


Alright! The transcription can definitely be recognized from our prediction, but it is not perfect yet. Training the model a bit longer, spending more time on the data preprocessing, and especially using a language model for decoding would certainly improve the model's overall performance.

For a demonstration model on a low-resource language, the results are quite acceptable however 🤗.